In [1]:
from calendar import c
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bayesianLearner import BayesianLearner

In [2]:
dataset = pd.read_csv("data/Data_for_UCI_named.csv")
# The p1 attribute is a non predictive one (calculated based on the other p attributes)
dataset = dataset.drop('p1', axis=1)
# Drop the other target column
dataset = dataset.drop('stab', axis=1)

learner = BayesianLearner(dataset)
learner.learn()
learner.evaluate(dataset)

Counter({'unstable': 6380, 'stable': 3620})
unstable    7097
stable      2903
dtype: int64
Predicted  stable  unstable
Labels                     
stable       2427      1193
unstable      476      5904
Correctly classified instances: 8331
incorrectly classified instances: 1669
=== Detailed Accuracy for class unstable ===
Accuracy: 0.8331
Precision: 0.8319008031562632
Recall: 0.9253918495297806
F measure: 0.8761593826519254
True Positive Rate: 0.9253918495297806
False Positive Rate: 0.32955801104972376
K-Coefficent 0.6225030692830136
=== Detailed Accuracy for class stable ===
Accuracy: 0.8331
Precision: 0.836031691353772
Recall: 0.6704419889502763
F measure: 0.7441361336808217
True Positive Rate: 0.6704419889502763
False Positive Rate: 0.07460815047021943
K-Coefficent 0.6225030692830136


,0,1,2,3,4,5,6
unstable,0.8331,0.831901,0.925392,0.876159,0.925392,0.329558,0.622503
stable,0.8331,0.836032,0.670442,0.744136,0.670442,0.074608,0.622503


In [3]:

datasetIris = pd.read_csv("data/iris.csv")
learner = BayesianLearner(datasetIris)
learner.learn()
learner.evaluate(datasetIris)

Counter({'Setosa': 50, 'Versicolor': 50, 'Virginica': 50})
Setosa        50
Versicolor    50
Virginica     50
dtype: int64
Predicted   Setosa  Versicolor  Virginica
Labels                                   
Setosa          50           0          0
Versicolor       0          47          3
Virginica        0           3         47
Correctly classified instances: 144
incorrectly classified instances: 6
=== Detailed Accuracy for class Setosa ===
Accuracy: 0.96
Precision: 1.0
Recall: 0.8928571428571429
F measure: 0.9433962264150945
True Positive Rate: 0.8928571428571429
False Positive Rate: 0.0
K-Coefficent 0.9126213592233009
=== Detailed Accuracy for class Versicolor ===
Accuracy: 0.96
Precision: 0.94
Recall: 0.94
F measure: 0.94
True Positive Rate: 0.94
False Positive Rate: 0.03
K-Coefficent 0.9099999999999999
=== Detailed Accuracy for class Virginica ===
Accuracy: 0.96
Precision: 0.94
Recall: 0.94
F measure: 0.94
True Positive Rate: 0.94
False Positive Rate: 0.03
K-Coefficent 0.9099999

,0,1,2,3,4,5,6
Setosa,0.96,1.0,0.892857,0.943396,0.892857,0.0,0.912621
Versicolor,0.96,0.94,0.94,0.94,0.94,0.03,0.91
Virginica,0.96,0.94,0.94,0.94,0.94,0.03,0.91


In [4]:
def crossValidate(dataset, groupNumber, verbose):
    dataset = dataset.sample(frac=1).reset_index(drop=True)
    if groupNumber <= 1:
        raise Exception("Set at least 2 cross-validation groups")
    
    cvPartition = dataset.shape[0] // groupNumber
    sumRes = None
    for i in range(groupNumber):        
        initialRow = i * cvPartition
        finalRow = (i + 1) * cvPartition
        testSet = dataset[initialRow:finalRow].reset_index(drop=True)
        trainingSet = dataset.drop(dataset.index[initialRow:finalRow], axis = 0, inplace = False).reset_index(drop=True)
        cvLearner = BayesianLearner(trainingSet)
        cvLearner.learn()
        if sumRes is None:
            sumRes = pd.DataFrame(0, index=cvLearner.classNames, columns=range(7))
        if verbose:
            print("*** Results for group", i,"***")
        res = cvLearner.evaluate(testSet, verbose)
        for cl in cvLearner.classNames:
            for j in range(7):
                sumRes.at[cl, j] += res.at[cl,j]
        avgRes = sumRes / groupNumber
    print("### Average of per-group results ###")
    for cl in cvLearner.classNames:
        print("=== Detailed Accuracy for class", cl, "===")
        print("Accuracy:", round(avgRes.at[cl, 0], 3))
        print("Precision:", round(avgRes.at[cl, 1], 3))
        print("Recall:", round(avgRes.at[cl, 2], 3))
        print("F measure:", round(avgRes.at[cl, 3], 3))
        print("True Positive Rate:", round(avgRes.at[cl, 4], 3))
        print("False Positive Rate:", round(avgRes.at[cl, 5], 3))
        print("K-Coefficent", round(avgRes.at[cl, 6], 3))
        

crossValidate(dataset, 10, False)

### Average of per-group results ###
=== Detailed Accuracy for class Virginica ===
Accuracy: 0.953
Precision: 0.922
Recall: 0.925
F measure: 0.918
True Positive Rate: 0.925
False Positive Rate: 0.033
K-Coefficent 0.879
=== Detailed Accuracy for class Versicolor ===
Accuracy: 0.953
Precision: 0.925
Recall: 0.93
F measure: 0.924
True Positive Rate: 0.93
False Positive Rate: 0.039
K-Coefficent 0.89
=== Detailed Accuracy for class Setosa ===
Accuracy: 0.953
Precision: 1.0
Recall: 0.882
F measure: 0.927
True Positive Rate: 0.882
False Positive Rate: 0.0
K-Coefficent 0.888
